In [1]:
from ladle.ladle import Ladle
from dataclasses import dataclass
import requests
import fitz  # PyMuPDF
import time
import os
import json
from selenium.webdriver.common.by import By
# Google Cloud Storage
from google.oauth2 import service_account
from google.api_core.exceptions import NotFound, Conflict, Forbidden, BadRequest
from google.cloud import storage
gcp_credentials_info = os.getenv("GCP_SERVICE_ACCOUNT_CREDENTIALS")
gcp_credentials_info = json.loads(gcp_credentials_info)
gcp_service_account_credentials = service_account.Credentials.from_service_account_info(gcp_credentials_info)
storage_client = storage.Client(credentials=gcp_service_account_credentials)

In [2]:
ladle = Ladle(headless=False, load_images=True)

In [4]:
CONTENT_TYPE_MAP = {
    # Documents
    "pdf": "application/pdf",
    "doc": "application/msword",
    "docx": "application/vnd.openxmlformats-officedocument.wordprocessingml.document",
    "xls": "application/vnd.ms-excel",
    "xlsx": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
    "ppt": "application/vnd.ms-powerpoint",
    "pptx": "application/vnd.openxmlformats-officedocument.presentationml.presentation",
    "txt": "text/plain",
    "rtf": "application/rtf",
    "csv": "text/csv",

    # Data / Markup
    "xml": "application/xml",
    "xsd": "application/xml",
    "json": "application/json",
    "yaml": "application/x-yaml",
    "yml": "application/x-yaml",

    # Archives / Compression
    "zip": "application/zip",
    "gz": "application/gzip",
    "tar": "application/x-tar",
    "rar": "application/vnd.rar",
    "7z": "application/x-7z-compressed",

    # Web
    "html": "text/html",
    "htm": "text/html",
    "css": "text/css",
    "js": "application/javascript",
    "jsonld": "application/ld+json",

    # Images
    "png": "image/png",
    "jpg": "image/jpeg",
    "jpeg": "image/jpeg",
    "gif": "image/gif",
    "bmp": "image/bmp",
    "svg": "image/svg+xml",
    "webp": "image/webp",
    "ico": "image/x-icon",

    # Audio / Video
    "mp3": "audio/mpeg",
    "wav": "audio/wav",
    "ogg": "audio/ogg",
    "mp4": "video/mp4",
    "mov": "video/quicktime",
    "avi": "video/x-msvideo",
    "webm": "video/webm",
}


In [5]:
def _get_bucket(storage_client, bucket_name):
    """
    Get GCS bucket, create if it doesn't exist
    """
    # Try to get the bucket — this avoids an explicit .exists() check
    try:
        bucket = storage_client.get_bucket(bucket_name)
    except NotFound:
        print(f"Bucket '{bucket_name}' not found. Creating it...")
        try:
            bucket = storage_client.create_bucket(bucket_name)
            print(f"✅ Created bucket '{bucket_name}'")
        except Conflict:
            # Another process might have created it simultaneously
            bucket = storage_client.get_bucket(bucket_name)
        except Forbidden:
            raise PermissionError(
                f"🚫 Cannot create bucket '{bucket_name}'. ",
                f"Your service account needs 'roles/storage.admin' permission."
            )
    return bucket

def _normalize_folder(folder):
    """
    Normalize folder path for GCS upload
    """
    normalized_folder = None
    if folder:
        candidate = folder.strip("/").replace("\\", "/")
        normalized_folder = candidate or None
    return normalized_folder

def _truncate_name(blob_name, normalized_folder, separator="_____"):
    """
    Truncate the blob name to fit within GCS limits (1024 BYTES).
    """
    max_bytes = 1024
    # Calculate folder overhead in BYTES
    folder_overhead = len(normalized_folder.encode('utf-8')) + 1 if normalized_folder else 0
    available_bytes = max_bytes - folder_overhead
    separator_bytes = len(separator.encode('utf-8'))
    if available_bytes <= separator_bytes:
        raise ValueError("Folder path too long to create object name.")
    blob_name_bytes = blob_name.encode('utf-8')
    # If it already fits, return as-is
    if len(blob_name_bytes) <= available_bytes:
        return blob_name
    # Calculate suffix length in bytes
    desired_suffix_bytes = min(100, len(blob_name_bytes))
    max_suffix_bytes = available_bytes - separator_bytes - 1
    if max_suffix_bytes <= 0:
        raise ValueError("Folder path too long to create object name.")
    suffix_bytes = min(desired_suffix_bytes, max_suffix_bytes)
    base_bytes = available_bytes - separator_bytes - suffix_bytes
    if base_bytes <= 0:
        raise ValueError("Folder path too long to create object name.")
    # Truncate at byte boundaries and decode safely
    base_part = blob_name_bytes[:base_bytes].decode('utf-8', errors='ignore')
    suffix_part = blob_name_bytes[-suffix_bytes:].decode('utf-8', errors='ignore')
    truncated_name = f"{base_part}{separator}{suffix_part}"
    return truncated_name

In [6]:
def upload_to_storage(storage_client, bucket_name, pdf_obj, folder=None):
    """
    Upload the PDF object to Google Cloud Storage.
    If the bucket doesn't exist, try to create it (without triggering a 403 from bucket.exists()).
    Skip the upload when the document is already stored.
    Optionally upload inside a folder within the bucket.
    """
    bucket = _get_bucket(storage_client, bucket_name)
    content_type = CONTENT_TYPE_MAP.get(pdf_obj["extension"], "application/file")
    blob_name = pdf_obj["name"].replace("/", "_").replace("\\", "_")
    normalized_folder = _normalize_folder(folder)

    try:
        blob_path = f"{normalized_folder}/{blob_name}" if normalized_folder else blob_name
        if bucket.get_blob(blob_path) is not None:
            # print(f"ℹ️ Skipping upload; '{blob_path}' already exists in gs://{bucket_name}")
            return

        blob = bucket.blob(blob_path)
        
        blob.upload_from_string(pdf_obj["bytes"], content_type=content_type)
        print(f"✅ Uploaded '{blob_path}' to gs://{bucket_name}")
    except BadRequest as err:
        err_text = str(err)
        if ("maximum object length") in err_text or ("The bucket name and object name together must be at most 1087 characters") in err_text:
            truncated_name = _truncate_name(blob_name, normalized_folder, separator="_____")
            truncated_path = (
                f"{normalized_folder}/{truncated_name}" if normalized_folder else truncated_name
            )
            # print(f"⚠️ Blob name too long; retrying as '{truncated_path}'")
            # print("Len: " + str(len(truncated_path)))
            if bucket.get_blob(truncated_path) is not None:
                print(f"ℹ️ Skipping upload; '{truncated_path}' already exists in gs://{bucket_name}")
                return
            fallback_blob = bucket.blob(truncated_path)
            fallback_blob.upload_from_string(pdf_obj["bytes"], content_type=content_type)
            # print(f"✅ Uploaded '{truncated_path}' to gs://{bucket_name}")
        else:
            raise

In [42]:
ladle = Ladle(headless=False)

# Agenzia entrate
### Provvedimenti non soggetti a pubblicità

In [43]:
main_folder = 'provvedimenti_non_soggetti_a_pubblicita'
bucket_name = "loomy-public-documents"

In [ ]:
ladle.driver.get("https://www.agenziaentrate.gov.it/portale/archivio/normativa-prassi-archivio-documentazione/provvedimenti/altri-provvedimenti-non-soggetti")
try:
    ladle.clicks.click('//*[@id="closePopup"]', timeout=2)
except Exception as e:
    print(f"Site opened with no popup showing")

possible_year_list_xpaths = [
    '/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[*]/div/div/section/div/div[2]/div/div/div/div/p/a',
    '/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[*]/div/div/section/div/div[2]/div/div/div/div/p/a'
]

# RACCOGLIE LISTA DEGLI ANNI
for year_list_xp in possible_year_list_xpaths:
    try:
        year_elements = ladle.elements.elements(year_list_xp)
        break  # success
    except Exception:
        continue

for i in range(7, len(year_elements)): # ANNI
    year = ladle.elements.element(f'/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[{2+i}]/div/div/section/div/div[2]/div/div/div/div/p/a', text=True)
    print("Year: " + year)
    # Save current page URL to return after visiting subsection. Years page.
    years_page_url = ladle.driver.current_url
    ladle.clicks.click(f'/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[{2+i}]/div/div/section/div/div[2]/div/div/div/div/p/a')

    # Re-capture elements on the new page (avoid stale references)
    possible_month_list_xpaths = [
        "/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/p/a",
        "/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/p/a",
        "/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[*]/div/div/section/div/div[2]/div/div/div/div/p/a",
        '/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[*]/div/div/section/div/div[2]/div/div/div/p/a'
        # "/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[*]/div/div/section/div/div[2]/div/div/div/div/p/a"
    ]
    # RACCOGLIE LISTA DEI MESI
    for month_list_xp in possible_month_list_xpaths:
        print("Trying month list xpath: " + month_list_xp)
        try:
            month_elements = ladle.elements.elements(month_list_xp)
            break  # success
        except Exception:
            continue

    for j in range(len(month_elements)): # MESI
        # Save current inner page URL to return after visiting detail link. Months page.
        months_list_page_url = ladle.driver.current_url
        
        month = None
        possible_month_xpaths = [xp.replace("*", str(1+j)) for xp in possible_month_list_xpaths]
        for month_xp in possible_month_xpaths:
            print("Trying single month xpath: " + month_xp)
            try:
                month = ladle.elements.element(month_xp, timeout=3, text=True)
                ladle.clicks.click(month_xp, timeout=3)
                break  # success
            except Exception:
                continue

        if month is None:
            # Go with div[2+j]
            possible_month_xpaths = [xp.replace("*", str(2+j)) for xp in possible_month_list_xpaths]
            print("Month is None")
            for month_xp in possible_month_xpaths:
                print("Trying single month xpath: " + month_xp)
                try:
                    month = ladle.elements.element(month_xp, timeout=3, text=True)
                    ladle.clicks.click(month_xp, timeout=3)
                    break  # success
                except Exception:
                    continue

        possible_doc_list_xpaths = [
            '/html/body/div[4]/div/div/div/div[3]/div/div/div/div[2]/div/div[2]/div/div/section/div/div[2]/div/div[*]/div/p/a',
            '/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[3]/div/div/section/div/div[2]/div/div[*]/div/p/a',
            '/html/body/div[1]/div/div/div/div[3]/div/div/div/div[2]/div/div[2]/div/div/section/div/div[2]/div/div[*]/div/p/a',
            '/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/div/p/a',
            '/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[3]/div/div/section/div/div[2]/div/div[*]/div/p/a',
            '/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/div/p/a',
            '/html/body/div[4]/div/div/div/div[3]/div/div/div/div[2]/div/div[1]/div/div/section/div/div[2]/div/div/div/div/p[1]/a[2]'
        ]

        print("Month: " + str(month))

        # RACCOGLIE DOCUMENTI NEL MESE
        for doc_list_xp in possible_doc_list_xpaths:
            try:
                elements = ladle.elements.elements(doc_list_xp)
                # Skip if takes the first element which is Articolo di legge ceh prevede la pubblicazione dei doc
                if len(elements) <= 1: 
                    continue
                else:
                    break  # success
            except Exception:
                continue
        
        if elements is None or len(elements) == 0:
            # No documents found for this month, continue to next month
            ladle.driver.get(years_page_url)
            continue

        for h in range(len(elements)): # DOCUMENTI NEL MESE
            possible_doc_xpaths = [xp.replace("*", str(1+h)) for xp in possible_doc_list_xpaths]
            for xp in possible_doc_xpaths:
                try:
                    doc_elem = ladle.elements.element(xp, timeout=2)
                    doc_xpath = xp  # Save the successful xpath for later use
                    break  # success
                except Exception:
                    continue
            
            time.sleep(0.5)  # Small delay to ensure text is loaded otherwise it opens the pdf
            doc_elem = ladle.elements.element(doc_xpath, timeout=2) # Re-fetch to avoid stale reference
            doc_elem_text = doc_elem.text
            # If not pdf it's a folder
            if 'pdf' in doc_elem_text.lower(): # can't be .endswith('pdf') because some pdfs have additional info at the end
                doc_url = doc_elem.get_attribute('href')
                pdf_resp = requests.get(doc_url)
                pdf_bytes = pdf_resp.content
                file_extension = pdf_resp.url.split('.')[-1].split('?')[0].split('/')[0].lower()

                # assemble python object
                pdf_obj = {
                    "url": doc_url,
                    "name": doc_elem_text,
                    "bytes": pdf_bytes,
                    "extension": file_extension
                }
                # Load the doc to gcp storage
                upload_to_storage(storage_client=storage_client,
                                  bucket_name=bucket_name,
                                  pdf_obj=pdf_obj,
                                  folder=f"{main_folder}/{year}/{month}")
            else:
                # Save current doc list page URL to return to after visiting the folder
                doc_list_page_url = ladle.driver.current_url

                ladle.clicks.click(doc_xpath, retries=5)

                # Explore folder contents
                # ! ul[*] sections can be more than 1, but for simplicty we only explore the first one which usually contains the main docs
                possible_doc_list_inside_folder_xpaths = [
                    '/html/body/div[4]/div/div/div/div[3]/div/div/div/div[2]/div/div/div/div/section/div/div[2]/div/div/div[2]/div/div/ul[1]/li[*]/a',
                    '/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div/div/div/section/div/div[2]/div/div/div[2]/div/div/ul/li[*]/a',
                    '/html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/div/p/a',
                    '/html/body/div[1]/div/div/div/div[3]/div/div/div/div[2]/div/div/div/div/section/div/div[2]/div/div/div[2]/div/div/ul[1]/li[*]/a',
                    '/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div/div/div/section/div/div[2]/div/div/div[2]/div/div/ul/li[*]/a',
                    '/html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/div/p/a'
                ]
                for doc_list_inside_folder_xp in possible_doc_list_inside_folder_xpaths:
                    try:
                        doc_list_inside_folder = ladle.elements.elements(doc_list_inside_folder_xp)
                        break  # success
                    except Exception:
                        continue

                for k in range(len(doc_list_inside_folder)): # DOCUMENTI NELLA CARTELLA
                    # Save current folder_doc list page URL to return to after visiting the docs into the clicked folder
                    folder_doc_list_page_url = ladle.driver.current_url

                    possible_doc_inside_folder_xpaths = [xp.replace("*", str(1+k)) for xp in possible_doc_list_inside_folder_xpaths]
                    for doc_inside_folder_xp in possible_doc_inside_folder_xpaths:
                        try:
                            folder_doc_elem = ladle.elements.element(doc_inside_folder_xp, timeout=2)
                            folder_doc_xpath = doc_inside_folder_xp  # Save the successful xpath for later use
                            break  # success
                        except Exception:
                            continue
                    
                    time.sleep(0.5)  # Small delay to ensure text is loaded otherwise it opens the pdf
                    folder_doc_elem = ladle.elements.element(folder_doc_xpath, timeout=2) # Re-fetch to avoid stale reference
                    folder_doc_elem_text = folder_doc_elem.text
                    # print("\t\t" +folder_doc_elem_text)

                    folder_doc_url = folder_doc_elem.get_attribute('href')
                    folder_pdf_resp = requests.get(folder_doc_url)
                    folder_pdf_bytes = folder_pdf_resp.content

                    # assemble python object
                    folder_pdf_obj = {
                        "url": folder_doc_url,
                        "name": f"{doc_elem_text} - {folder_doc_elem_text}",
                        "bytes": folder_pdf_bytes,
                        "extension": folder_pdf_resp.url.split('.')[-1].split('?')[0].split('/')[0].lower()
                    }
                    # Load the doc to gcp storage
                    upload_to_storage(storage_client=storage_client,
                                        bucket_name=bucket_name,
                                        pdf_obj=folder_pdf_obj,
                                        folder=f"{main_folder}/{year}/{month}")
                
                # Return to doc list page
                ladle.driver.get(doc_list_page_url)

        # Return to inner list using URL instead of back
        ladle.driver.get(months_list_page_url)

    # Return to outer list using URL instead of back
    ladle.driver.get(years_page_url)

Year: 2018 - Provvedimenti del Direttore non soggetti a pubblicità legale
Trying month list xpath: /html/body/div[4]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/p/a
Trying month list xpath: /html/body/div[1]/div/div/div/div[3]/div/div/div[3]/main/div[2]/div/div/section/div/div[2]/div/div[*]/p/a


KeyboardInterrupt: 

In [70]:
len(pdf_obj['name'].encode('utf-8').decode('utf-8', errors='ignore'))

1006

In [15]:
len("provvedimenti_con_pubblicità/2025 - Provvedimenti del Direttore soggetti a pubblicità legale/Febbraio 2025")

106

In [ ]:
@dataclass
class DocumentRecord:
    source_type: str
    doc_id: str
    title: str
    url: str
    collected_at: str
    text_path: str
    meta: Dict[str, Any] = field(default_factory=dict)

    def to_dict(self):
        d = asdict(self)
        return d

# Sentenze cassazione

In [26]:
ladle = Ladle(headless=False, load_images=True)

In [27]:
ladle.driver.get("https://www.italgiure.giustizia.it/sncass/")

In [28]:
from datetime import datetime
anno_corrente = datetime.now().year
button_mapping = {
    "archivio": {
        "CIVILE": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[2]/td/div/div/div/div/div/table/tbody/tr[1]/td[2]",
        "PENALE": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[2]/td/div/div/div/div/div/table/tbody/tr[2]/td[2]"
    },
    "tipo": {
        "Decreto": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[3]/td/div/div/div/div/div/table/tbody/tr[1]/td[2]",
        "Ordinanza": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[3]/td/div/div/div/div/div/table/tbody/tr[2]/td[2]",
        "Ordinanza Interlocutoria": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[3]/td/div/div/div/div/div/table/tbody/tr[3]/td[2]",
        "Sentenza": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[3]/td/div/div/div/div/div/table/tbody/tr[4]/td[2]"
    },
    "sezione": {
        "PRIMA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[1]/td[2]",
        "SECONDA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[2]/td[2]",
        "TERZA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[3]/td[2]",
        "QUARTA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[4]/td[2]",
        "QUINTA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[5]/td[2]",
        "SESTA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[6]/td[2]",
        "SETTIMA": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[7]/td[2]",
        "FERIALE": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[8]/td[2]",
        "LAVORO": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[9]/td[2]",
        "UNITE": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[4]/td/div/div/div/div/div/table/tbody/tr[10]/td[2]",
    },
    "anno": { # Max 5 anni + corrente
        f"{anno_corrente}": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[5]/td/div/div/div/div/div/table/tbody/tr[1]/td[2]",
        f"{anno_corrente - 1}": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[5]/td/div/div/div/div/div/table/tbody/tr[2]/td[2]",
        f"{anno_corrente - 2}": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[5]/td/div/div/div/div/div/table/tbody/tr[3]/td[2]",
        f"{anno_corrente - 3}": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[5]/td/div/div/div/div/div/table/tbody/tr[4]/td[2]",
        f"{anno_corrente - 4}": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[5]/td/div/div/div/div/div/table/tbody/tr[5]/td[2]",
        f"{anno_corrente - 5}": "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[1]/table/tbody/tr[5]/td/div/div/div/div/div/table/tbody/tr[6]/td[2]"
    }
}

In [29]:
ladle.clicks.click(button_mapping['sezione']['PRIMA'])
time.sleep(3)
ladle.clicks.click(button_mapping['sezione']['TERZA'])
time.sleep(3)
ladle.clicks.click(button_mapping['sezione']['QUINTA'])
time.sleep(3)
ladle.clicks.click(button_mapping['sezione']['LAVORO'])
time.sleep(3)
# L'archivio va cliccato per ultimo perché la CIVILE elimina alcune sezioni e perciò riordina gli xpath e.g. td[5] -> td[4]
ladle.clicks.click(button_mapping['archivio']['CIVILE'])

prova sotto

In [ ]:
max_retries = 10
new_page_button_xpath = "/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[2]/div/div[2]/div[11]/span"
# While new_page_button_xpath in the source page click the next page button
new_page_button = ladle.elements.element(new_page_button_xpath)
new_page = ladle.elements.element_displayed(element=new_page_button)

while new_page:
    new_page_button = ladle.elements.element(new_page_button_xpath)
    new_page = ladle.elements.element_displayed(xpath=new_page_button_xpath)
    
    # Scrape page pdfs
    pdf_list = ladle.elements.elements('/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[2]/div/div[2]/div[*]/div/h3/a')
    for n in range(len(pdf_list)):
        ladle.clicks.click(f'/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[2]/div/div[2]/div[{n+1}]/div/h3/a')
        time.sleep(1)
        print(ladle.driver.current_url)
        input()
        if 'captcha' in ladle.driver.page_source: # Not working because does not see the html of the newly opened page
        # TODO: fix the check for captcha on new page
            print("Captcha detected, waiting 30 seconds...")
            time.sleep(30)
            input()
        # get pdf url
        doc_url = ladle.driver.current_url
        if '.pdf' in doc_url.lower():
            pdf_resp = requests.get(doc_url)
            pdf_bytes = pdf_resp.content
            file_extension = pdf_resp.url.split('.')[-1].split('?')[0].split('/')[0].lower()

            # assemble python object
            pdf_obj = {
                "url": doc_url,
                "name": ladle.elements.element('/html/body/div[1]/div[3]/form/table/tbody[2]/tr/td[2]/div/div[1]/h2', text=True),
                "bytes": pdf_bytes,
                "extension": file_extension
            }
            # Load the doc to gcp storage
            upload_to_storage(storage_client=storage_client,
                              bucket_name=bucket_name,
                              pdf_obj=pdf_obj,
                              folder=f"italgiure/temp/sezione_civile/prima_terza_quinta_lavoro")

    # pdf_obj = {
    #     "url": doc_url,
    #     "bytes": pdf_content['pdf_bytes'],
    #     "text": pdf_content['pdf_text']
    # }
    # print(pdf_obj['url'])
    
    ladle.clicks.click(new_page_button_xpath)

https://www.italgiure.giustizia.it/sncass/
https://www.italgiure.giustizia.it/sncass/
https://www.italgiure.giustizia.it/sncass/
